**Settings**

In [ ]:
!pip install transformers --quiet # package installer for python

     |████████████████████████████████| 1.5MB 8.5MB/s 
     |████████████████████████████████| 2.9MB 30.7MB/s 
     |████████████████████████████████| 890kB 57.5MB/s 


In [ ]:
#git에 올려둔 friends 데이터셋 받아오기
!wget https://raw.githubusercontent.com/DEOK-IL/KU-NLP-2020-2/master/Friends/friends_dev.json
!wget https://raw.githubusercontent.com/DEOK-IL/KU-NLP-2020-2/master/Friends/friends_test.json
!wget https://raw.githubusercontent.com/DEOK-IL/KU-NLP-2020-2/master/Friends/friends_train.json

--2020-12-25 13:23:51--  https://raw.githubusercontent.com/DEOK-IL/KU-NLP-2020-2/master/Friends/friends_dev.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 229392 (224K) [text/plain]
Saving to: ‘friends_dev.json’

friends_dev.json    100%[===================>] 224.02K  --.-KB/s    in 0.02s   

2020-12-25 13:23:52 (10.5 MB/s) - ‘friends_dev.json’ saved [229392/229392]

--2020-12-25 13:23:52--  https://raw.githubusercontent.com/DEOK-IL/KU-NLP-2020-2/master/Friends/friends_test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 544341 (

In [ ]:
import tensorflow as tf
import torch
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import random
import time
import datetime
import json

In [ ]:
def jsonToDf(file_name):
  with open(file_name, encoding = 'utf-8', mode = 'r') as file:
    json_array = json.load(file)
  
  result = pd.DataFrame.from_dict(json_array[0])

  is_first = True
  for array in json_array:
    if is_first:
      is_first = False
      continue
    
    temp_df = pd.DataFrame.from_dict(array)
    result = result.append(temp_df, ignore_index = True)

  return result

In [ ]:
train = jsonToDf('friends_train.json')

**Test Dataset 업로드/전처리**

In [ ]:
#캐글 테스트셋 다운로드
!wget https://raw.githubusercontent.com/DEOK-IL/KU-NLP-2020-2/master/en_data.csv

--2020-12-25 13:23:59--  https://raw.githubusercontent.com/DEOK-IL/KU-NLP-2020-2/master/en_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97634 (95K) [text/plain]
Saving to: ‘en_data.csv’

en_data.csv         100%[===================>]  95.35K  --.-KB/s    in 0.009s  

2020-12-25 13:24:00 (10.5 MB/s) - ‘en_data.csv’ saved [97634/97634]



In [ ]:
#테스트 데이터 파일 입력, 별도 검증 데이터 사용 시 해당 파일 명으로 대체
test_data = 'en_data.csv'

In [ ]:
test = pd.read_csv(test_data)

In [ ]:
test.head()

,id,i_dialog,i_utterance,speaker,utterance
0,0,0,0,Phoebe,"Alright, whadyou do with him?"
1,1,0,1,Monica,Oh! You're awake!
2,2,0,2,Joey,Then you gotta come clean with Ma! This is not...
3,3,0,3,Mr. Tribbiani,"Yeah, but this is"
4,4,0,4,Joey,I don't wanna hear it! Now go to my room!


In [ ]:
print(test.shape)

(1623, 5)


In [ ]:
MAX_LEN = 85
def getInputsFromTest(dataset):
  data = dataset.copy(deep=True)
  #data['utterance'] = data['utterance'].str.lower()

  utterances = data['utterance']
  utterances = ["[CLS] " + str(utterance) + " [SEP]" for utterance in utterances]
  
  tokenizer = BertTokenizer.from_pretrained('bert-large-cased', do_lower_case=False)
  tokenized_texts = [tokenizer.tokenize(utterance) for utterance in utterances]

  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

  return input_ids, attention_masks

In [ ]:
def getIndex(dataset):
  data = dataset.copy(deep = True)
  input_index = data.id.tolist()
  return torch.tensor(input_index)

In [ ]:
test_inputs, test_masks = getInputsFromTest(test)

test_index = getIndex(test)
test_inputs = torch.tensor(test_inputs)
test_masks = torch.tensor(test_masks)

In [ ]:
batch_size = 32

test_data = TensorDataset(test_index, test_inputs, test_masks)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


**모델 다운로드**

In [ ]:
#미리 학습해둔 모델 다운로드
!wget https://www.dropbox.com/s/bhke3hobsdwznln/model_eng_electra_large.pt?dl=0

--2020-12-25 13:24:01--  https://www.dropbox.com/s/bhke3hobsdwznln/model_eng_electra_large.pt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/bhke3hobsdwznln/model_eng_electra_large.pt [following]
--2020-12-25 13:24:01--  https://www.dropbox.com/s/raw/bhke3hobsdwznln/model_eng_electra_large.pt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucab9fc10e9b076f5e93f9a0b1e5.dl.dropboxusercontent.com/cd/0/inline/BFt2v5PayuQeXvClpYUlvc4ZVs9Jf_WtVQmCsXS1Iaz8PvNOx_nSrBW6GeJ--MAtw8SW1E_uS_wp_Vco7OvUy0AjxBFTEkF4YEdfLNuFYHV5EYBakGa6ympKz09jBG2l-GM/file# [following]
--2020-12-25 13:24:02--  https://ucab9fc10e9b076f5e93f9a0b1e5.dl.dropboxusercontent.com/cd/0/inline/BFt2v5PayuQeXvClpYUlvc4ZVs9Jf_WtVQmCsXS1Iaz8PvNOx_nSrBW6GeJ--MAt

In [ ]:
model = ElectraForSequenceClassification.from_pretrained('google/electra-large-generator', num_labels=8)
model.load_state_dict(torch.load("model_eng_electra_large.pt?dl=0"))
model.cuda()

Some weights of the model checkpoint at google/electra-large-generator were not used when initializing ElectraForSequenceClassification: ['generator_predictions.LayerNorm.weight', 'generator_predictions.LayerNorm.bias', 'generator_predictions.dense.weight', 'generator_predictions.dense.bias', 'generator_lm_head.weight', 'generator_lm_head.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-generator and are newly initializ

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=1024, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linea

**테스트셋 검증 및 결과 데이터 생성**

In [ ]:
tmp_test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=1)
test_result = test.copy(deep = True)
test_result = test_result.drop(columns = ['i_dialog', 'i_utterance', 'speaker'])
test_result['Predicted'] = 'default'

encoder = LabelEncoder()
labels = train['emotion'].values
encoder.fit(labels)
labels = encoder.transform(labels)


for step, batch in enumerate(tmp_test_dataloader):
    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_index, b_input_ids, b_input_mask = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    idx = b_index.item()
    test_result['Predicted'][idx] = encoder.classes_[np.argmax(logits)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
test_result.tail()

,id,utterance,Predicted
1618,1618,Nooo.,neutral
1619,1619,"Hi, Kate!",neutral
1620,1620,"Hi, Lauren.",neutral
1621,1621,"Hi, Lauren.",neutral
1622,1622,"Hi, pig!",neutral


In [ ]:
test_result = test_result.drop(columns = ['utterance'])

In [ ]:
test_csv = test_result.to_csv('2019516021_이덕일_sample.csv', index=False)

In [ ]:
#파일 다운로드
from google.colab import files

files.download('2019516021_이덕일_sample.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>